<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/prueba_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prueba de modelos

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 3.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
from transformers import pipeline

In [25]:
examples = ["hay que matarlos a todos", "Vamos a la plaza a marchar","Los chinos son todos sucios","Culpa de los bolitas"]

### Modelo 1 BETO binario

Parametros:

output_dir=repo_name,
eval_strategy ="steps",
adam_beta1=0.9,
adam_beta2=0.98,
eval_steps=10000,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
weight_decay=0.01,
push_to_hub=True,

In [26]:
specific_model = pipeline(model="natdebandi/beto_hate_speech_ar_bin")
specific_model(examples)

[{'label': 'LABEL_1', 'score': 0.9779065847396851},
 {'label': 'LABEL_0', 'score': 0.9912819862365723},
 {'label': 'LABEL_1', 'score': 0.9802582263946533},
 {'label': 'LABEL_1', 'score': 0.9858723282814026}]

### Modelo 2 BETO multietiqueta




In [27]:

checkpoint = "natdebandi/beto_hate_speech_ar_multiclasif"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

tokens = tokenizer(examples, padding=True, truncation=True, return_tensors="pt")
outputs = model(**tokens)

outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 3.8660,  5.1885, -4.4542, -3.0403, -3.0044, -4.1031, -5.7982, -6.0370,
         -4.5832,  1.3988],
        [-6.6326, -8.3743, -8.4561, -8.7309, -7.9588, -8.0575, -8.0021, -8.8995,
         -8.7085, -7.3292],
        [ 5.2895, -5.2991, -4.8129, -4.4569,  3.7987, -5.3095, -5.3135, -4.3695,
          1.9925, -5.5745],
        [ 4.4619, -6.2723, -5.2999, -5.9450,  4.0760, -6.7237, -5.9405, -6.1795,
         -5.9148, -6.5303]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [28]:
model.config.id2label

{0: 'HATEFUL',
 1: 'CALLS',
 2: 'WOMEN',
 3: 'LGBTI',
 4: 'RACISM',
 5: 'CLASS',
 6: 'POLITICS',
 7: 'DISABLED',
 8: 'APPEARANCE',
 9: 'CRIMINAL'}

In [29]:
logits = outputs.logits
logits.shape

torch.Size([4, 10])

In [30]:
import numpy as np

# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1


In [31]:
predictions

array([[1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [32]:
# turn predicted id's into actual label names
predicted_labels = [model.config.id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

##NO PUDE METER MAS DE UN TEXTO

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()